Importing libraries

In [311]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import re
from json.decoder import JSONDecodeError
import datetime

In [312]:
import import_ipynb
import ETL 

In [313]:
films = ETL.Films()
credits_final = films.get_credits_final()
movies_final = films.get_movies_final()

Starting ...


Falta graficas, borrar los que tienen valor de 0 en las votaciones y popularidad

In [314]:
movies_final.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,vote_count,belongs_to_collection_json,production_companies_json,release_date_clean,release_date_clean_format,retorno,month_movies,day_movies,year_movies,key_title
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",30/10/1995,...,5415,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'name': 'Pixar Animation Studios', 'id': 3}]",30/10/1995,30/10/1995,12.451801,10,lunes,1995,toy story
1,,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",15/12/1995,...,2413,,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",15/12/1995,15/12/1995,4.043035,12,viernes,1995,jumanji


In [315]:
movies_final.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'belongs_to_collection_json', 'production_companies_json',
       'release_date_clean', 'release_date_clean_format', 'retorno',
       'month_movies', 'day_movies', 'year_movies', 'key_title'],
      dtype='object')

In [316]:
movies_final['vote_count'].describe()

count    8427.000000
mean      201.836953
std       595.855008
min         0.000000
25%        12.000000
50%        35.000000
75%       126.000000
max      9678.000000
Name: vote_count, dtype: float64

In [317]:
movies_final['popularity'].describe()

count    8427.000000
mean        5.280933
std         4.984210
min         0.000000
25%         1.523100
50%         4.063135
75%         7.925636
max       140.950236
Name: popularity, dtype: float64

In [318]:
movies_final['spoken_languages'].describe()


count                                         8427
unique                                         572
top       [{'iso_639_1': 'en', 'name': 'English'}]
freq                                          6066
Name: spoken_languages, dtype: object

In [319]:
movies_final['vote_average'].mean()

np.float64(6.103548119140856)

In [320]:
#Replace NaN with an empty string
movies_final['overview'] = movies_final['overview'].fillna('')

Obtener las peliculas en base al titulo, ver las similitud

In [321]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')



#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(movies_final['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


(8427, 28545)

In [322]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)



In [323]:
cosine_sim.shape

(8427, 8427)

In [324]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(movies_final.index, index=movies_final['title']).drop_duplicates()


In [325]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [326]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations_by_title(title, cosine_sim=cosine_sim):
    idx = indices[title]
    

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar movies
    sim_scores = sim_scores[1:6]


    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_final['title'].iloc[movie_indices]


In [327]:
get_recommendations_by_title('Jumanji')

6166           Brainscan
8801             Quintet
9503           Word Wars
6055    Poolhall Junkies
2486            eXistenZ
Name: title, dtype: object

Recomendaciones donde interviene el producto entre popularidad y y votos

In [328]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [329]:
movies_recomendacion = movies_final
def create_references(ref):
    string =  str(ref['popularity']*ref['vote_count'])
    
    return string


movies_recomendacion['reference'] = movies_recomendacion.apply(create_references, axis=1)
movies_recomendacion.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,belongs_to_collection_json,production_companies_json,release_date_clean,release_date_clean_format,retorno,month_movies,day_movies,year_movies,key_title,reference
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",30/10/1995,...,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'name': 'Pixar Animation Studios', 'id': 3}]",30/10/1995,30/10/1995,12.451801,10,lunes,1995,toy story,118842.696345


In [330]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
count_matrix = count.fit_transform(movies_recomendacion['reference'])

In [331]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)



In [332]:
# Reset index of your main DataFrame and construct reverse mapping as before
movies_recomendacion = movies_recomendacion.reset_index()
indices = pd.Series(movies_recomendacion.index, index=movies_recomendacion['title'])

In [333]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations_by_prod_ref(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie    
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_recomendacion['title'].iloc[movie_indices]

In [334]:
get_recommendations_by_prod_ref('Jumanji',cosine_sim2)

0                      Toy Story
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
5                           Heat
Name: title, dtype: object

Recomendaciones por weighted_rating

In [335]:
q_movies = movies_final


In [336]:
C = q_movies['vote_average'].mean()
m = q_movies['vote_count'].quantile(0.90)


In [337]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [338]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [339]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(5)



,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358,8.5,8.379116
834,The Godfather,6024,8.5,8.335494
2843,Fight Club,9678,8.3,8.203653
292,Pulp Fiction,8670,8.3,8.192997
522,Schindler's List,4436,8.3,8.100159


In [340]:
index = q_movies.index
q_movies['index']= index

In [341]:
movieRatings = q_movies.pivot_table(index=['vote_count', 'vote_average'], columns='title',values='score', aggfunc='sum', fill_value=0)  
movieRatings.head()

title                    'night, Mother  ...All the Marbles  \
vote_count vote_average                                       
0          0.0                      0.0                 0.0   
1          0.0                      0.0                 0.0   
           0.5                      0.0                 0.0   
           1.0                      0.0                 0.0   
           2.0                      0.0                 0.0   

title                    ...And God Created Woman  ...And Justice for All  \
vote_count vote_average                                                     
0          0.0                                0.0                     0.0   
1          0.0                                0.0                     0.0   
           0.5                                0.0                     0.0   
           1.0                                0.0                     0.0   
           2.0                                0.0                     0.0   

title                    ...tick... tick... tick...  1-900   10  \
vote_count vote_average                                           
0          0.0                                  0.0    0.0  0.0   
1          0.0                                  0.0    0.0  0.0   
           0.5                                  0.0    0.0  0.0   
           1.0                                  0.0    0.0  0.0   
           2.0                                  0.0    0.0  0.0   

title                    10 Rillington Place  10 Things I Hate About You  \
vote_count vote_average                                                    
0          0.0                           0.0                         0.0   
1          0.0                           0.0                         0.0   
           0.5                           0.0                         0.0   
           1.0                           0.0                         0.0   
           2.0                           0.0                         0.0   

title                    10 to Midnight  ...  Zorro, The Gay Blade  Zulu  \
vote_count vote_average                  ...                               
0          0.0                      0.0  ...                   0.0   0.0   
1          0.0                      0.0  ...                   0.0   0.0   
           0.5                      0.0  ...                   0.0   0.0   
           1.0                      0.0  ...                   0.0   0.0   
           2.0                      0.0  ...                   0.0   0.0   

title                    Zus & Zo  eXistenZ  ivans xtc.  xXx  \
vote_count vote_average                                        
0          0.0                0.0       0.0         0.0  0.0   
1          0.0                0.0       0.0         0.0  0.0   
           0.5                0.0       0.0         0.0  0.0   
           1.0                0.0       0.0         0.0  0.0   
           2.0                0.0       0.0         0.0  0.0   

title                    xXx: State of the Union  ¡Three Amigos!   Él  \
vote_count vote_average                                                 
0          0.0                               0.0             0.0  0.0   
1          0.0                               0.0             0.0  0.0   
           0.5                               0.0             0.0  0.0   
           1.0                               0.0             0.0  0.0   
           2.0                               0.0             0.0  0.0   

title                    ’Round Midnight  
vote_count vote_average                   
0          0.0                       0.0  
1          0.0                       0.0  
           0.5                       0.0  
           1.0                       0.0  
           2.0                       0.0  

[5 rows x 8217 columns]

In [342]:
movieRatings = movieRatings.applymap(lambda x: 1 if x > 0 else 0)

C:\Users\DELIA MONTOYA\AppData\Local\Temp\ipykernel_8892\3000525354.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  movieRatings = movieRatings.applymap(lambda x: 1 if x > 0 else 0)


In [343]:
movieRatings

title                    'night, Mother  ...All the Marbles  \
vote_count vote_average                                       
0          0.0                        0                   0   
1          0.0                        0                   0   
           0.5                        0                   0   
           1.0                        0                   0   
           2.0                        0                   0   
...                                 ...                 ...   
8358       8.5                        0                   0   
8670       8.3                        0                   0   
8892       8.0                        0                   0   
9079       7.9                        0                   0   
9678       8.3                        0                   0   

title                    ...And God Created Woman  ...And Justice for All  \
vote_count vote_average                                                     
0          0.0                                  0                       0   
1          0.0                                  0                       0   
           0.5                                  0                       0   
           1.0                                  0                       0   
           2.0                                  0                       0   
...                                           ...                     ...   
8358       8.5                                  0                       0   
8670       8.3                                  0                       0   
8892       8.0                                  0                       0   
9079       7.9                                  0                       0   
9678       8.3                                  0                       0   

title                    ...tick... tick... tick...  1-900  10  \
vote_count vote_average                                          
0          0.0                                    0      0   0   
1          0.0                                    0      0   0   
           0.5                                    0      0   0   
           1.0                                    0      0   0   
           2.0                                    0      0   0   
...                                             ...    ...  ..   
8358       8.5                                    0      0   0   
8670       8.3                                    0      0   0   
8892       8.0                                    0      0   0   
9079       7.9                                    0      0   0   
9678       8.3                                    0      0   0   

title                    10 Rillington Place  10 Things I Hate About You  \
vote_count vote_average                                                    
0          0.0                             0                           0   
1          0.0                             0                           0   
           0.5                             0                           0   
           1.0                             0                           0   
           2.0                             0                           0   
...                                      ...                         ...   
8358       8.5                             0                           0   
8670       8.3                             0                           0   
8892       8.0                             0                           0   
9079       7.9                             0                           0   
9678       8.3                             0                           0   

title                    10 to Midnight  ...  Zorro, The Gay Blade  Zulu  \
vote_count vote_average                  ...                               
0          0.0                        0  ...                     0     0   
1          0.0                        0  ...                     0     0   
           0.5            

In [344]:
from sklearn.metrics.pairwise import cosine_similarity


In [345]:
index_movieRatings = pd.DataFrame(
    cosine_similarity(movieRatings)
)

In [346]:
index_movieRatings

,0,1,2,3,4,5,6,7,8,9,...,4570,4571,4572,4573,4574,4575,4576,4577,4578,4579
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [347]:
index_movieRatings.columns = movieRatings.index
index_movieRatings['index'] = movieRatings.index
index_movieRatings = index_movieRatings.set_index('index')

In [348]:
index_movieRatings['index'] = movieRatings.index
index_movieRatings = index_movieRatings.set_index('index')

In [349]:
index_movieRatings

vote_count      0    1                                          ... 7191 7641  \
vote_average  0.0  0.0  0.5  1.0  2.0  2.5  3.0  4.0  4.5  5.0  ...  7.5  8.0   
index                                                           ...             
(0, 0.0)      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
(1, 0.0)      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
(1, 0.5)      0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
(1, 1.0)      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
(1, 2.0)      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
(8358, 8.5)   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
(8670, 8.3)   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
(8892, 8.0)   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
(9079, 7.9)   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
(9678, 8.3)   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

vote_count   7770 8147 8226 8358 8670 8892 9079 9678  
vote_average  7.5  8.2  8.1  8.5  8.3  8.0  7.9  8.3  
index                                                 
(0, 0.0)      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
(1, 0.0)      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
(1, 0.5)      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
(1, 1.0)      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
(1, 2.0)      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...           ...  ...  ...  ...  ...  ...  ...  ...  
(8358, 8.5)   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  
(8670, 8.3)   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  
(8892, 8.0)   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
(9079, 7.9)   0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  
(9678, 8.3)   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  

[4580 rows x 4580 columns]

In [350]:
toyStoryRatings = movieRatings['Frida']
#print(toyStoryRatings)

# Correlamos el resto de peliculas (columnas) con la seleccionada (toy story)  
similarMovies = movieRatings.corrwith(toyStoryRatings)  
#print(similarMovies)
similarMovies = similarMovies.dropna()  
#print(similarMovies)
df = pd.DataFrame(similarMovies)

# Las ordenamos por el valor de score que hemos generado, de forma descendente  
similarMovies.sort_values(ascending=False)  

title
The Bridges of Madison County    1.000000
Frida                            1.000000
The Wild Thornberrys Movie      -0.000218
Yellowbeard                     -0.000218
Twin Town                       -0.000218
                                   ...   
Wuthering Heights               -0.000378
King Solomon's Mines            -0.000378
Little Women                    -0.000378
The Phantom of the Opera        -0.000378
Hamlet                          -0.000437
Length: 8217, dtype: float64

Recomendación basada en créditos, géneros y palabras clave

In [373]:
metadata['spoken_languages']

0        []
1        []
2        []
3        []
4        []
         ..
10026    []
10028    []
10029    []
10030    []
10032    []
Name: spoken_languages, Length: 8427, dtype: object

In [424]:
metadata = movies_final
metadata = metadata.drop(['budget','production_companies','production_countries','release_date_clean','revenue','runtime','status', 'tagline','release_date_clean', 'release_date_clean_format', 'retorno',
       'month_movies', 'day_movies', 'year_movies','popularity','release_date','vote_average','vote_count','belongs_to_collection_json','production_companies_json','title','reference','score','belongs_to_collection'
       ,'original_language'], axis=1)

In [425]:
def get_list(x):    
    if isinstance(x, list): 
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list        
        if len(names) > 3:
            
            names = names[:3]
        return names
    else:
        x=x.replace("'","\"")
        data = json.loads(x)
        names = [i['name'] for i in data]
        return names
    #Return empty list in case of missing/malformed data
    return []

In [426]:
features = ['spoken_languages', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [427]:
metadata

,genres,id,overview,spoken_languages,key_title
0,"[Animation, Comedy, Family]",862,"Led by Woody, Andy's toys live happily in his ...",[English],toy story
1,"[Adventure, Fantasy, Family]",8844,When siblings Judy and Peter discover an encha...,"[English, Français]",jumanji
2,"[Romance, Comedy]",15602,A family wedding reignites the ancient feud be...,[English],grumpier old men
3,"[Comedy, Drama, Romance]",31357,"Cheated on, mistreated and stepped on, the wom...",[English],waiting to exhale
4,[Comedy],11862,Just when George Banks has recovered from his ...,[English],father of the bride part ii
...,...,...,...,...,...
10026,"[Thriller, Drama, Romance]",43344,"Francisco is rich, rather strict on principles...",[Español],él
10028,[Drama],133750,"While Erendira, a beautiful teenage girl, has ...","[Português, Español]",eréndira
10029,"[Drama, Romance]",16902,This period drama frames the tumultuous affair...,[English],the private lives of elizabeth and essex
10030,"[Drama, War]",41131,"In post war Germany, a dislocated and orphaned...","[Český, English, Français, Deutsch, עִבְרִית, ...",the search


In [487]:
def create_mix(x):
    spoken_languages = ' '.join(x['spoken_languages'])
    genres = ' '.join(x['genres'])
    return spoken_languages + ' '+genres+' '+x['overview'] + ' '

In [488]:
# Create a new soup feature
metadata['mix'] = metadata.apply(create_mix, axis=1)

In [490]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['mix'])

In [493]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [495]:
# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['key_title'])

In [499]:
get_recommendations('frida', cosine_sim2)

1251                          Kissed
4555                       Bar Girls
8152                This Girl's Life
2268    The Loss of Sexual Innocence
3606                           Alfie
Name: title, dtype: object

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación de películas. El EDA debería incluir gráficas interesantes para extraer datos, como por ejemplo una nube de palabras con las palabras más frecuentes en los títulos de las películas. Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score de similaridad y devolverá una lista de Python con 5 valores, cada uno siendo el string del nombre de las películas con mayor puntaje, en orden descendente. Debe ser deployado como una función adicional de la API anterior y debe llamarse: